# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096924


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:04<01:06,  2.36s/it]

Rendering models:  10%|█         | 3/30 [00:07<01:08,  2.53s/it]

Rendering models:  13%|█▎        | 4/30 [00:08<00:51,  1.99s/it]

Rendering models:  17%|█▋        | 5/30 [00:09<00:40,  1.61s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:17<00:01,  2.62it/s]

Rendering models:  93%|█████████▎| 28/30 [00:18<00:00,  2.56it/s]

Rendering models:  97%|█████████▋| 29/30 [00:18<00:00,  2.80it/s]

Rendering models: 100%|██████████| 30/30 [00:19<00:00,  2.21it/s]

not-logged-in-3af001450ab9bcf73c06    0.000588
Jnursssmith5263                       0.000822
sorenjensen01                         0.000364
jnarayanbvg                           0.000543
imkSushi                              0.000370
not-logged-in-e7932b1588574fe4dae2    0.000562
not-logged-in-c6917d9b19f41e6c2995    0.000855
framosuw                              0.000656
not-logged-in-a5072c52bbdaf4fd1d17    0.303671
not-logged-in-7fb560ed89c1075b73d2    0.014940
Snoopmatt                             0.020673
not-logged-in-7e969a324309f376dde8    0.000804
not-logged-in-468edef11712cfc1e24c    0.005002
not-logged-in-5f30b2bb9b7f9d49da41    0.004358
alexismfernandez                      0.000689
lstallard                             0.001737
TRISTANM18                            0.000585
pangeli5                              0.000419
not-logged-in-9f30f411df9d28d2c541    0.001691
Lavadude                              0.019975
not-logged-in-68c93388f62f1185dba6    0.000495
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())